In [13]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.worldfootball.net/all_matches/eng-premier-league-2023-2024/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

table = soup.find("table", class_="standard_tabelle")
rows = []

if table is not None:  # Check if the table was found
    for row in table.find_all('tr'):  # Use find_all to iterate through all rows
        title_cell = row.find('td')  # Find the first cell in the row
        if title_cell is not None:
            title = title_cell.get_text(strip=True)
            rows.append(title)
        else:
            rows.append('No title')
else:
    rows.append('Table not found')

rows


No title
11/08/2023
12/08/2023





13/08/2023

14/08/2023
No title
18/08/2023
19/08/2023





20/08/2023

21/08/2023
No title
25/08/2023
26/08/2023





27/08/2023


No title
01/09/2023
02/09/2023





03/09/2023


No title
16/09/2023






17/09/2023

18/09/2023
No title
23/09/2023





24/09/2023



No title
30/09/2023







01/10/2023
02/10/2023
No title
07/10/2023






08/10/2023


No title
21/10/2023






22/10/2023

23/10/2023
No title
27/10/2023
28/10/2023




29/10/2023



No title
04/11/2023









No title
11/11/2023









No title
25/11/2023









No title
02/12/2023









No title
05/12/2023








06/12/2023
No title
09/12/2023









No title
16/12/2023









No title
23/12/2023









No title
26/12/2023









No title
30/12/2023









No title
13/01/2024









No title
30/01/2024







31/01/2024

No title
03/02/2024









No title
10/02/2024









No title
17/02/2024









No title
24/02/2024









No title
02/03/2024








In [4]:
headers

['1. Round',
 '2. Round',
 '3. Round',
 '4. Round',
 '5. Round',
 '6. Round',
 '7. Round',
 '8. Round',
 '9. Round',
 '10. Round',
 '11. Round',
 '12. Round',
 '13. Round',
 '14. Round',
 '15. Round',
 '16. Round',
 '17. Round',
 '18. Round',
 '19. Round',
 '20. Round',
 '21. Round',
 '22. Round',
 '23. Round',
 '24. Round',
 '25. Round',
 '26. Round',
 '27. Round',
 '28. Round',
 '29. Round',
 '30. Round',
 '31. Round',
 '32. Round',
 '33. Round',
 '34. Round',
 '35. Round',
 '36. Round',
 '37. Round',
 '38. Round']